In [5]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union

def run_with_config(
    script_path: str,
    class_or_function: Optional[str] = None,
    method: Optional[str] = None,
    base_dir: str = '.',
    config_filename: str = 'config.json',
    **kwargs
) -> int:
    """
    Универсальная функция для запуска Python-скрипта с параметрами через конфигурационный файл.
    
    Args:
        script_path: Путь к Python-скрипту, который нужно запустить
        class_or_function: Имя класса или функции для вызова (опционально)
        method: Имя метода класса для вызова (для классов)
        base_dir: Базовая директория для сохранения конфигурационного файла
        config_filename: Имя файла конфигурации
        **kwargs: Любые параметры, которые нужно передать в скрипт
        
    Returns:
        Код возврата процесса (0 - успешное выполнение)
    """
    # Конвертируем все datetime объекты в строки
    processed_kwargs = {}
    for k, v in kwargs.items():
        if isinstance(v, datetime.date) or isinstance(v, datetime.datetime):
            processed_kwargs[k] = v.isoformat().split('T')[0]  # Формат YYYY-MM-DD
        else:
            processed_kwargs[k] = v
    
    # Создаем конфигурацию
    config = {
        "params": processed_kwargs
    }
    
    # Добавляем информацию о классе и методе, если указаны
    if class_or_function:
        config["target"] = class_or_function
    if method:
        config["method"] = method
    
    # Полный путь к конфигурационному файлу
    config_path = os.path.join(base_dir, config_filename)
    
    # Сохраняем конфигурацию в файл
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"Конфигурация сохранена в {config_path}")
    
    # Проверяем существование скрипта
    if not os.path.exists(script_path):
        raise FileNotFoundError(f"Скрипт {script_path} не найден")
    
    # Запускаем скрипт с указанием файла конфигурации
    command = [sys.executable, script_path, '--config', config_path]
    
    print("Запуск команды:", " ".join(command))
    
    # Запускаем процесс и перенаправляем вывод
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, 
                              universal_newlines=True, bufsize=1)
    
    # Выводим результаты в реальном времени
    for line in process.stdout:
        print(line, end='')
    
    # Ждем завершения процесса
    process.wait()
    
    if process.returncode == 0:
        print("\nПроцесс успешно завершен")
    else:
        print(f"\nОшибка при выполнении процесса, код: {process.returncode}")
    
    return process.returncode


In [6]:
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import requests
import time
import zipfile
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys')

#####

import market_data
importlib.reload(market_data)
from market_data import MarketDataManager

######

import fundamental_data
importlib.reload(fundamental_data)
from fundamental_data import FinamFinancialParser

######

import tech_analysis
importlib.reload(tech_analysis)
from tech_analysis import TechAnalysisPipeline


######

import news_pipeline
importlib.reload(news_pipeline)
from news_pipeline import NewsPipeline

######

import data_integration
importlib.reload(data_integration)
from data_integration import DataIntegrator

## Globals

In [3]:
token = "t.GVEetX09IVL9qECDUhjThfpg5TrE2Oxw9cUnAIOxcHh35M2sv1YMy9LBfkt73F3WlV_Ik08etZOVrUNI8StLBQ"
kursach_directory_path = "/Users/aeshef/Documents/GitHub/kursach"
your_directory_path = "/Users/aeshef/Documents/GitHub/kursach/data"

tickers = pd.read_csv(f"{your_directory_path}/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'VTBR',  # ВТБ
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2024-06-01"
end_date="2025-04-15"
timeframe="1h"

YOUR_API_ID = 28994010
YOUR_API_HASH = "6e7a57fdfe1a10b0a3434104b42badf2"

## Initial parser

In [4]:
manager = MarketDataManager(token, your_directory_path)

multi_ticker_data = manager.get_data_for_multiple_tickers(
    tickers=tickers_list, 
    start_date=start_date, 
    end_date=end_date, 
    timeframe=timeframe
)

Congratulations! BBG004731489 is a correct figi for GMKN
Congratulations! BBG004730N88 is a correct figi for SBER
Congratulations! BBG004731354 is a correct figi for ROSN
Congratulations! BBG004731032 is a correct figi for LKOH
Congratulations! BBG004730RP0 is a correct figi for GAZP
GMKN: No data found
Congratulations! BBG004731489 is a correct figi for GMKN
ROSN: No data found
Congratulations! BBG004731354 is a correct figi for ROSN
LKOH: No data found
Congratulations! BBG004731032 is a correct figi for LKOH
GAZP: No data found
Congratulations! BBG004730RP0 is a correct figi for GAZP

GMKN data statistics:
  - Total rows: 8582
  - First date: 2025-04-06 00:00:00
  - Last date: 2025-04-14 20:49:00
Congratulations! BBG004RVFFC0 is a correct figi for TATN
SBER: No data found
Congratulations! BBG004730N88 is a correct figi for SBER

ROSN data statistics:
  - Total rows: 9541
  - First date: 2025-04-06 00:00:00
  - Last date: 2025-04-14 20:49:00
Congratulations! BBG004S681W1 is a correct 

## Fundamental

In [5]:
# import logging

# for ticker in tickers_list:
#     logging.info(f"\n========== Processing {ticker} ==========\n")
#     parser = FinamFinancialParser(ticker)   # ПЕРЕДАЁШЬ тикер!
#     financial_data_df = parser.parse_financial_data()
#     if financial_data_df is not None:
#         parser.save_to_csv(financial_data_df)
#         logging.info(f"Финансовые данные для {ticker} сохранены успешно")
#     else:
#         logging.warning(f"No data found for {ticker}. Проверь логи, html и soup-просмотр!")

## Tech

In [19]:
pipeline = TechAnalysisPipeline(base_dir="/Users/aeshef/Documents/GitHub/kursach")
pipeline.run_pipeline()

2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Инициализирован TechAnalysisPipeline.
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Инициализирован TechAnalysisPipeline.
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-22 15:07:39 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-22 15:07:39 - TechPipeline - INFO - Начало загрузки данных.
2025-04-22 15:07:40 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_tech_indicator

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-22 15:07:40 - TechPipeline - INFO - Начало загрузки данных.
2025-04-22 15:07:40 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_tech_indicators.csv
2025-04-22 15:07:4

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-22 15:07:40 - TechPipeline - INFO - Начало загрузки данных.
2025-04-22 15:07:40 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-22 15:07:40 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/LKOH/tech_analysis/LKOH_tech_indicators.csv
2025-04-22 15:07:4

...

2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-22 15:07:45 - TechPipeline - INFO - Начало загрузки данных.
2025-04-22 15:07:45 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-22 15:07:45 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_tech_indicators.csv
2025-04-22 15:07:4

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-22 15:07:46 - TechPipeline - INFO - Начало загрузки данных.
2025-04-22 15:07:46 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_tech_indicators.csv
2025-04-22 15:07:4

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===
2025-04-22 15:07:46 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===


## News

In [7]:
import datetime

run_with_config(
    script_path="/Users/aeshef/Documents/GitHub/kursach/pys/news_pipeline.py",
    class_or_function="NewsPipeline",
    method="run_pipeline",
    base_dir=kursach_directory_path,
    config_filename="news_pipeline_config.json",
    
    tickers=tickers_list,
    collect_telegram=True,
    telegram_api_id=YOUR_API_ID,
    telegram_api_hash=YOUR_API_HASH,
    telegram_channel="cbrstocks",
    telegram_limit=10000,
    start_date=start_date,
    end_date=end_date,
    use_cached_telegram=True
)

Конфигурация сохранена в /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск команды: /Library/Developer/CommandLineTools/usr/bin/python3 /Users/aeshef/Documents/GitHub/kursach/pys/news_pipeline.py --config /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск pipeline.run_pipeline с параметрами: {'tickers': ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'VTBR', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL'], 'collect_telegram': True, 'telegram_api_id': 28994010, 'telegram_api_hash': '6e7a57fdfe1a10b0a3434104b42badf2', 'telegram_channel': 'cbrstocks', 'telegram_limit': 10000, 'start_date': datetime.date(2024, 6, 1), 'end_date': datetime.date(2025, 4, 15), 'use_cached_telegram': True}
2025-04-22 14:44:20 - NewsPipeline - INFO - Запуск пайплайна анализа новостей для 17 тикеров
2025-04-22 14:44:20 - NewsPipeline - INFO - Диапазон дат: 2024-06-01 - 2025-04-15
2025-04-22 14:44:20 - pymorphy2.opencorpora_dict.wrapper - 

0

In [7]:
integrator = DataIntegrator(base_path='/Users/aeshef/Documents/GitHub/kursach/data/processed_data', nan_fill_method='zero')
combined_data = integrator.run_pipeline(tickers=tickers_list, output_path = '/Users/aeshef/Documents/GitHub/kursach/data/df.csv')

2025-04-22 15:15:17 - DataIntegrator - INFO - Обработка тикера SBER...
2025-04-22 15:15:17 - DataIntegrator - INFO - Данные для SBER загружены за период: 2024-06-01 - 2025-04-14
2025-04-22 15:15:17 - DataIntegrator - INFO -   - Обнаружено 1186 пропущенных значений для SBER
2025-04-22 15:15:17 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-22 15:15:17 - DataIntegrator - INFO - Обработка тикера GAZP...
2025-04-22 15:15:17 - DataIntegrator - INFO - Данные для GAZP загружены за период: 2024-06-01 - 2025-04-14
2025-04-22 15:15:17 - DataIntegrator - INFO -   - Обнаружено 1050 пропущенных значений для GAZP
2025-04-22 15:15:17 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-22 15:15:17 - DataIntegrator - INFO - Обработка тикера LKOH...
2025-04-22 15:15:17 - DataIntegrator - INFO - Данные для LKOH загружены за период: 2024-06-01 - 2025-04-14
2025-04-22 15:15:17 - DataIntegrator - INFO -   - Обнаружено 1050 пропущенных значений

## Экспы

In [9]:
# d = {}

# for ticker in tickers_list:
#     df = pd.read_csv(f'/Users/aeshef/Documents/GitHub/kursach/data/processed_data/{ticker}/news_analysis/{ticker}_news_with_sentiment.csv')
#     # только нейтральные новости
#     df = df[df["sentiment_category"] == "neutral"]
#     # исключаем тексты с "Фокус недели"
#     df = df[~df["clean_text"].str.contains("Фокус недели", na=False)]
#     df = df[~df["clean_text"].str.contains("Что было сегодня", na=False)]
#     df = df[~df["clean_text"].str.contains("Что было вчера", na=False)]
#     df = df[~df["clean_text"].str.contains("акция от Шпиона РЦБ", na=False)]
#     df = df[~df["clean_text"].str.contains("Закрытые идеи Мозгового центра Сигналов РЦБ", na=False)]
#     df = df[~df["clean_text"].str.contains("Вопрос от подписчик", na=False)]
#     # df = df[~df["clean_text"].str.contains("Главное за неделю", na=False)]
#     d[ticker] = df["clean_text"]

In [10]:
# for key in d.keys():
#     print(f"ТИКЕР: {key}")
#     sample = d[key].sample(15)  # random 15 записей из DataFrame/Series d[key]
#     for index, news in sample.items():  # если это Series; если DataFrame — for _, row in sample.iterrows():
#         print(news)
#     print()
#     print("===========")
#     print()